In [1]:
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
train_df = pd.read_csv("../../input/feedback-prize-english-language-learning/train.csv")
train_df.head(2)

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5


In [3]:
model_path_list = [
    #('../../07_Embeddings/exp/result/07_v1_01/oof_df.csv',1), # svr
    
    #('../../14_Baseline4/exp/result/14_v1_01/oof_df.csv',1), # deberta-v3-base
    ('../../14_Baseline4/exp/result/14_v1_07/oof_df.csv',1), # deberta-v3-base, 10folds     
    #('../../14_Baseline4/exp/result/14_v1_08/oof_df.csv',1), # deberta-base
    #('../../14_Baseline4/exp/result/14_v1_09/oof_df.csv',1), # roberta-base
    ('../../14_Baseline4/exp/result/14_v1_10/oof_df.csv',1), # deberta-v3-large
    ('../../14_Baseline4/exp/result/14_v1_11/oof_df.csv',1), # xlm-roberta-base
    #('../../14_Baseline4/exp/result/14_v1_12/oof_df.csv',1), # deberta-large
    #('../../14_Baseline4/exp/result/14_v1_13/oof_df.csv',1), # roberta-large
    #('../../14_Baseline4/exp/result/14_v1_14/oof_df.csv',1), # xlm-roberta-large
#     ('../../14_Baseline4/exp/result/14_v1_15/oof_df.csv',1), # deberta-v2-xlarge
#     ('../../14_Baseline4/exp/result/14_v1_16/oof_df.csv',1), # deberta-xlarge
    
    ('../../14_Baseline4/exp/result/14_v2_01/oof_df.csv',1), # deberta-v3-base, seed200
    ('../../14_Baseline4/exp/result/14_v2_02/oof_df.csv',1), # deberta-v3-large, seed200
#     ('../../14_Baseline4/exp/result/14_v2_03/oof_df.csv',1), # deberta-base, seed200
    
    #('../../17_Pseudo3/exp/result/17_v1_01/oof_df.csv',1), # deberta-v3-base
    #('../../17_Pseudo3/exp/result/17_v1_02/oof_df.csv',1), # deberta-v3-base
    #('../../17_Pseudo3/exp/result/17_v1_03/oof_df.csv',1), # deberta-v3-large
    #('../../17_Pseudo3/exp/result/17_v1_04/oof_df.csv',1), # deberta-v3-base, seed200
    #('../../17_Pseudo3/exp/result/17_v1_05/oof_df.csv',1), # deberta-base
    #('../../17_Pseudo3/exp/result/17_v1_06/oof_df.csv',1), # deberta-large
    #('../../17_Pseudo3/exp/result/17_v1_07/oof_df.csv',1), # deberta-v3-small
    
    #('../../18_Pseudo4/exp/result/18_v1_01/oof_df.csv',1), # deberta-v3-base
    #('../../18_Pseudo4/exp/result/18_v1_02/oof_df.csv',1), # deberta-v3-large
    #('../../18_Pseudo4/exp/result/18_v1_03/oof_df.csv',1), # deberta-v3-small
    
    #('../../19_Distillation/exp/result/19_v1_01/oof_df.csv',1), # deberta-v3-base <- dbv3-l
    ('../../19_Distillation/exp/result/19_v1_02/oof_df.csv',1), # deberta-v3-base <- dbv2-xl
    ('../../19_Distillation/exp/result/19_v1_03/oof_df.csv',1), # deberta-v3-base <- db-xl
    #('../../19_Distillation/exp/result/19_v1_04/oof_df.csv',1), # deberta-v3-small <- dbv3-l
    #('../../19_Distillation/exp/result/19_v1_05/oof_df.csv',1), # deberta-v3-base <- db-l
    
    # single models
    ('../../20_Distillation2/exp/result/20_v2_01/oof_df.csv',1), # deberta-v3-base <- dbv3-b
    #('../../20_Distillation2/exp/result/20_v2_02/oof_df.csv',1), # deberta-v3-base <- dbv3-l
    #('../../20_Distillation2/exp/result/20_v2_03/oof_df.csv',1), # deberta-v3-large <- dbv3-l
    #('../../20_Distillation2/exp/result/20_v2_04/oof_df.csv',1), # deberta-base <- db-b
    #('../../20_Distillation2/exp/result/20_v2_05/oof_df.csv',1), # deberta-v3-base <- db-b
    #('../../20_Distillation2/exp/result/20_v2_06/oof_df.csv',1), # deberta-v3-base <- db-l
    #('../../20_Distillation2/exp/result/20_v2_07/oof_df.csv',1), # deberta-v3-base <- svr
    #('../../20_Distillation2/exp/result/20_v2_08/oof_df.csv',1), # deberta-v3-base <- xlm-rb-b
    #('../../20_Distillation2/exp/result/20_v2_09/oof_df.csv',1), # deberta-v3-base <- xlm-rb-l
    #('../../20_Distillation2/exp/result/20_v2_10/oof_df.csv',1), # deberta-v3-base <- dbv2-xl
    #('../../20_Distillation2/exp/result/20_v2_11/oof_df.csv',1), # deberta-v3-base <- dbv3-b, 10folds
]

oof_df_list = [
    pd.read_csv(model_path) for model_path,_ in model_path_list
]
weights_list = [
    w for _,w in model_path_list
]
weights_list = [w/sum(weights_list) for w in weights_list]

for oof_df in oof_df_list:
    oof_df = train_df[['text_id']].merge(oof_df, how='left', on='text_id')
    
num_models = len(model_path_list)

TARGET_COLS = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']

preds = 0
for oof_df, w in zip(oof_df_list, weights_list):
    preds += oof_df[TARGET_COLS].values * w

oofs = []
for oof_df in oof_df_list:
    oofs.append(oof_df[TARGET_COLS].values)

In [4]:
oof_df = oof_df_list[0].copy()
for i,col in enumerate(TARGET_COLS):
    oof_df[col] = preds[:,i]

In [5]:
import numpy as np

def calc_metric(pred, gt):
    '''
    pred : (num_data, num_labels)
    gt : (num_data, num_labels)
    '''
    score = np.sqrt(np.mean((pred - gt)**2, axis=0))
    score = score.mean()
    return score

In [6]:
score = calc_metric(pred=oof_df[TARGET_COLS].values, gt=train_df[TARGET_COLS].values)
print('CV={:.4f}'.format(score))

CV=0.4434


# Optimize class-wise

In [7]:
ensemble_predictions=np.stack(oofs)
ensemble_predictions.shape

(8, 3911, 6)

In [8]:
from skopt import gp_minimize

def run_optimize(i):
    target_col = TARGET_COLS[i]
    print("*"*50)
    print(target_col)
    print("*"*50)
    def ensemble_score(weights,return_pred=False):
        weights=np.array(weights)
        weights=weights.reshape(-1,1,1)/weights.sum()
        p=weights.reshape(-1,1,1)*ensemble_predictions[:,:,i,None]
        p=p.sum(0)
        score=calc_metric(p, train_df[[target_col]])
        if return_pred:
            return score,p
        else:
            return score   
    results = gp_minimize(ensemble_score, np.array([[0.1,1] for i in range(len(ensemble_predictions))]),
                          verbose=True,random_state=2022)
    best_weights=np.array(results['x'])/sum(results['x'])
    print(best_weights)
    # smaller is better for this metric
    score, ensemble_pred=ensemble_score(best_weights,True)
    print("score={:.4f}", score)
    return score, best_weights

In [9]:
scores = {}
best_weights = {}
for i,target_col in enumerate(TARGET_COLS):
    score, weights = run_optimize(i)
    scores[target_col] = score
    best_weights[target_col] = weights

**************************************************
cohesion
**************************************************
Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0056
Function value obtained: 0.4728
Current minimum: 0.4728
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0028
Function value obtained: 0.4719
Current minimum: 0.4719
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0019
Function value obtained: 0.4722
Current minimum: 0.4719
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.0019
Function value obtained: 0.4724
Current minimum: 0.4719
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.

Iteration No: 41 ended. Search finished for the next optimal point.
Time taken: 0.3685
Function value obtained: 0.4715
Current minimum: 0.4710
Iteration No: 42 started. Searching for the next optimal point.
Iteration No: 42 ended. Search finished for the next optimal point.
Time taken: 0.4225
Function value obtained: 0.4721
Current minimum: 0.4710
Iteration No: 43 started. Searching for the next optimal point.
Iteration No: 43 ended. Search finished for the next optimal point.
Time taken: 0.3861
Function value obtained: 0.4717
Current minimum: 0.4710
Iteration No: 44 started. Searching for the next optimal point.
Iteration No: 44 ended. Search finished for the next optimal point.
Time taken: 0.4262
Function value obtained: 0.4722
Current minimum: 0.4710
Iteration No: 45 started. Searching for the next optimal point.
Iteration No: 45 ended. Search finished for the next optimal point.
Time taken: 0.3865
Function value obtained: 0.4723
Current minimum: 0.4710
Iteration No: 46 started. Sea

Iteration No: 81 ended. Search finished for the next optimal point.
Time taken: 0.5731
Function value obtained: 0.4707
Current minimum: 0.4707
Iteration No: 82 started. Searching for the next optimal point.
Iteration No: 82 ended. Search finished for the next optimal point.
Time taken: 0.5980
Function value obtained: 0.4733
Current minimum: 0.4707
Iteration No: 83 started. Searching for the next optimal point.
Iteration No: 83 ended. Search finished for the next optimal point.
Time taken: 0.6113
Function value obtained: 0.4707
Current minimum: 0.4707
Iteration No: 84 started. Searching for the next optimal point.
Iteration No: 84 ended. Search finished for the next optimal point.
Time taken: 0.6155
Function value obtained: 0.4732
Current minimum: 0.4707
Iteration No: 85 started. Searching for the next optimal point.
Iteration No: 85 ended. Search finished for the next optimal point.
Time taken: 0.6005
Function value obtained: 0.4707
Current minimum: 0.4707
Iteration No: 86 started. Sea

Iteration No: 21 ended. Search finished for the next optimal point.
Time taken: 0.2906
Function value obtained: 0.4387
Current minimum: 0.4375
Iteration No: 22 started. Searching for the next optimal point.
Iteration No: 22 ended. Search finished for the next optimal point.
Time taken: 0.2805
Function value obtained: 0.4378
Current minimum: 0.4375
Iteration No: 23 started. Searching for the next optimal point.
Iteration No: 23 ended. Search finished for the next optimal point.
Time taken: 0.3042
Function value obtained: 0.4380
Current minimum: 0.4375
Iteration No: 24 started. Searching for the next optimal point.
Iteration No: 24 ended. Search finished for the next optimal point.
Time taken: 0.2803
Function value obtained: 0.4379
Current minimum: 0.4375
Iteration No: 25 started. Searching for the next optimal point.
Iteration No: 25 ended. Search finished for the next optimal point.
Time taken: 0.3074
Function value obtained: 0.4393
Current minimum: 0.4375
Iteration No: 26 started. Sea

Iteration No: 61 ended. Search finished for the next optimal point.
Time taken: 0.4878
Function value obtained: 0.4379
Current minimum: 0.4372
Iteration No: 62 started. Searching for the next optimal point.
Iteration No: 62 ended. Search finished for the next optimal point.
Time taken: 0.4932
Function value obtained: 0.4372
Current minimum: 0.4372
Iteration No: 63 started. Searching for the next optimal point.
Iteration No: 63 ended. Search finished for the next optimal point.
Time taken: 0.5125
Function value obtained: 0.4378
Current minimum: 0.4372
Iteration No: 64 started. Searching for the next optimal point.
Iteration No: 64 ended. Search finished for the next optimal point.
Time taken: 0.5489
Function value obtained: 0.4379
Current minimum: 0.4372
Iteration No: 65 started. Searching for the next optimal point.
Iteration No: 65 ended. Search finished for the next optimal point.
Time taken: 0.5724
Function value obtained: 0.4384
Current minimum: 0.4372
Iteration No: 66 started. Sea

...

Iteration No: 11 ended. Search finished for the next optimal point.
Time taken: 0.2412
Function value obtained: 0.4639
Current minimum: 0.4615
Iteration No: 12 started. Searching for the next optimal point.
Iteration No: 12 ended. Search finished for the next optimal point.
Time taken: 0.2005
Function value obtained: 0.4634
Current minimum: 0.4615
Iteration No: 13 started. Searching for the next optimal point.
Iteration No: 13 ended. Search finished for the next optimal point.
Time taken: 0.2137
Function value obtained: 0.4618
Current minimum: 0.4615
Iteration No: 14 started. Searching for the next optimal point.
Iteration No: 14 ended. Search finished for the next optimal point.
Time taken: 0.2892
Function value obtained: 0.4621
Current minimum: 0.4615
Iteration No: 15 started. Searching for the next optimal point.
Iteration No: 15 ended. Search finished for the next optimal point.
Time taken: 0.2522
Function value obtained: 0.4626
Current minimum: 0.4615
Iteration No: 16 started. Sea

Iteration No: 51 ended. Search finished for the next optimal point.
Time taken: 0.4966
Function value obtained: 0.4617
Current minimum: 0.4614
Iteration No: 52 started. Searching for the next optimal point.
Iteration No: 52 ended. Search finished for the next optimal point.
Time taken: 0.4957
Function value obtained: 0.4620
Current minimum: 0.4614
Iteration No: 53 started. Searching for the next optimal point.
Iteration No: 53 ended. Search finished for the next optimal point.
Time taken: 0.5041
Function value obtained: 0.4623
Current minimum: 0.4614
Iteration No: 54 started. Searching for the next optimal point.
Iteration No: 54 ended. Search finished for the next optimal point.
Time taken: 0.5087
Function value obtained: 0.4615
Current minimum: 0.4614
Iteration No: 55 started. Searching for the next optimal point.
Iteration No: 55 ended. Search finished for the next optimal point.
Time taken: 0.5862
Function value obtained: 0.4619
Current minimum: 0.4614
Iteration No: 56 started. Sea

Iteration No: 91 ended. Search finished for the next optimal point.
Time taken: 0.7278
Function value obtained: 0.4613
Current minimum: 0.4613
Iteration No: 92 started. Searching for the next optimal point.
Iteration No: 92 ended. Search finished for the next optimal point.
Time taken: 0.7543
Function value obtained: 0.4622
Current minimum: 0.4613
Iteration No: 93 started. Searching for the next optimal point.
Iteration No: 93 ended. Search finished for the next optimal point.
Time taken: 0.8205
Function value obtained: 0.4621
Current minimum: 0.4613
Iteration No: 94 started. Searching for the next optimal point.
Iteration No: 94 ended. Search finished for the next optimal point.
Time taken: 0.7554
Function value obtained: 0.4636
Current minimum: 0.4613
Iteration No: 95 started. Searching for the next optimal point.
Iteration No: 95 ended. Search finished for the next optimal point.
Time taken: 0.7651
Function value obtained: 0.4626
Current minimum: 0.4613
Iteration No: 96 started. Sea

Iteration No: 31 ended. Search finished for the next optimal point.
Time taken: 0.3227
Function value obtained: 0.4370
Current minimum: 0.4369
Iteration No: 32 started. Searching for the next optimal point.
Iteration No: 32 ended. Search finished for the next optimal point.
Time taken: 0.3290
Function value obtained: 0.4372
Current minimum: 0.4369
Iteration No: 33 started. Searching for the next optimal point.
Iteration No: 33 ended. Search finished for the next optimal point.
Time taken: 0.3741
Function value obtained: 0.4370
Current minimum: 0.4369
Iteration No: 34 started. Searching for the next optimal point.
Iteration No: 34 ended. Search finished for the next optimal point.
Time taken: 0.4084
Function value obtained: 0.4370
Current minimum: 0.4369
Iteration No: 35 started. Searching for the next optimal point.
Iteration No: 35 ended. Search finished for the next optimal point.
Time taken: 0.3590
Function value obtained: 0.4370
Current minimum: 0.4369
Iteration No: 36 started. Sea

Iteration No: 71 ended. Search finished for the next optimal point.
Time taken: 0.5460
Function value obtained: 0.4370
Current minimum: 0.4367
Iteration No: 72 started. Searching for the next optimal point.
Iteration No: 72 ended. Search finished for the next optimal point.
Time taken: 0.5705
Function value obtained: 0.4376
Current minimum: 0.4367
Iteration No: 73 started. Searching for the next optimal point.
Iteration No: 73 ended. Search finished for the next optimal point.
Time taken: 0.5992
Function value obtained: 0.4367
Current minimum: 0.4367
Iteration No: 74 started. Searching for the next optimal point.
Iteration No: 74 ended. Search finished for the next optimal point.
Time taken: 0.5621
Function value obtained: 0.4372
Current minimum: 0.4367
Iteration No: 75 started. Searching for the next optimal point.
Iteration No: 75 ended. Search finished for the next optimal point.
Time taken: 0.5958
Function value obtained: 0.4370
Current minimum: 0.4367
Iteration No: 76 started. Sea

In [10]:
best_weights

{'cohesion': array([0.27878332, 0.03705712, 0.02787833, 0.13920248, 0.27878332,
        0.05939974, 0.15101736, 0.02787833]),
 'syntax': array([0.24351614, 0.1898744 , 0.09924382, 0.02539242, 0.19051929,
        0.1833999 , 0.02435161, 0.04370242]),
 'vocabulary': array([0.23166781, 0.22516856, 0.11936578, 0.02655932, 0.2604841 ,
        0.02655932, 0.02655932, 0.08363578]),
 'phraseology': array([0.19354298, 0.16487681, 0.05180749, 0.14213281, 0.1502423 ,
        0.14073474, 0.09834668, 0.05831619]),
 'grammar': array([0.20501435, 0.23761811, 0.04939716, 0.15085155, 0.21317525,
        0.06291426, 0.05726751, 0.02376181]),
 'conventions': array([0.18280014, 0.12377418, 0.09026733, 0.08349764, 0.18280014,
        0.1357804 , 0.18280014, 0.01828001])}

In [11]:
scores

{'cohesion': 0.4707219988913814,
 'syntax': 0.4371618874969804,
 'vocabulary': 0.4043290238111619,
 'phraseology': 0.4458461656554935,
 'grammar': 0.46128015855327936,
 'conventions': 0.43673682749353876}

In [12]:
CV = sum(scores.values()) / len(scores)
print("CV={:.4f}".format(CV))

CV=0.4427


# Error Analysis - Check Corr

In [13]:
cols = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
train_df[cols].corr()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
cohesion,1.000000,0.695459,0.666151,0.690058,0.638689,0.666151
syntax,0.695459,1.000000,0.680562,0.725467,0.709525,0.700025
vocabulary,0.666151,0.680562,1.000000,0.735261,0.654852,0.664292
phraseology,0.690058,0.725467,0.735261,1.000000,0.719746,0.666842
grammar,0.638689,0.709525,0.654852,0.719746,1.000000,0.673301
conventions,0.666151,0.700025,0.664292,0.666842,0.673301,1.000000


In [14]:
cols = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
oof_df[cols].corr()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
cohesion,1.000000,0.952958,0.923312,0.920262,0.838896,0.894917
syntax,0.952958,1.000000,0.942049,0.964710,0.922240,0.930881
vocabulary,0.923312,0.942049,1.000000,0.962241,0.882328,0.895271
phraseology,0.920262,0.964710,0.962241,1.000000,0.947951,0.888377
grammar,0.838896,0.922240,0.882328,0.947951,1.000000,0.863446
conventions,0.894917,0.930881,0.895271,0.888377,0.863446,1.000000
